In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import time
from datetime import datetime, timedelta
from topaz import TopazAPI
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import requests
import os
import concurrent.futures
import math
# import mapping
import torch
import torch.nn as nn
import torch.nn.functional as F

In [41]:
data = pd.read_feather('./data/topaz_data_w_bsp_new6_nz.fth')
single_race = data.query('raceId == 914184484')

In [3]:
date = single_race.meetingDate.iloc[0]
date

Timestamp('2023-07-03 00:00:00+0000', tz='UTC')

In [4]:
single_day = data.query('meetingDate == @date')
single_day

,meetingDate,state,track,distance,raceId,raceTypeCode,raceNumber,boxNumber,rugNumber,runId,...,prev_race_track,prev_race_state,next_race,prev_BSP_price,prev_start_price,stats_topaz,dogid,raceid,stats_cols,track_hash
301162,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,1.0,1,915839614,...,Addington,NZ,918532419.0,55.000000,38.1,"[0.5339285731315613, 1.0, 1.0, 1.0, 0.0, 0.0, ...",549391123,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301163,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,2.0,2,915839615,...,Addington,NZ,920247464.0,14.500000,11.8,"[1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.1560566574335...",491585837,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301164,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,3.0,3,915839616,...,Addington,NZ,914237106.0,34.995636,21.9,"[0.7160714268684387, 3.0, 6.0, 0.0462962947785...",525211021,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301165,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,4.0,4,915839617,...,Addington,NZ,914237107.0,18.000000,13.8,"[0.02500000037252903, 4.0, 1.0, 0.027777777984...",638803603,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301166,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,6.0,6,915839619,...,Addington,NZ,918532429.0,17.000000,12.4,"[0.13214285671710968, 6.0, 7.0, 0.333333343267...",614057168,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301250,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,3.0,3,915839704,...,Addington,NZ,914186722.0,15.409545,11.1,"[0.1130952388048172, 3.0, 6.0, 1.0, 0.0, 0.0, ...",638803595,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301251,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,4.0,4,915839705,...,Addington,NZ,918532419.0,65.000000,29.4,"[0.5773809552192688, 4.0, 1.0, 0.8863636255264...",525210989,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301252,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,6.0,6,915839706,...,Addington,NZ,914237107.0,18.500000,12.4,"[0.2916666567325592, 6.0, 1.0, 0.2159090936183...",592709125,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301253,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,7.0,7,915839707,...,Ascot Park,NZ,914186229.0,11.500000,13.3,"[0.5297619104385376, 7.0, 1.0, 0.0, 0.0, 0.0, ...",556974762,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465


In [5]:
def add_classes(df):
    df_g = df.groupby('raceId')
    df_out = df.copy()
    for raceid,data in df_g:
        print(raceid,data.shape)
        place = torch.tensor(data['place'].values)
        place_inv = 1/place
        margin = torch.tensor(data['resultMargin'].values)
        onehot = torch.tensor(data['win'].values)
        bsp = torch.tensor(data['BSP'].values)

        place_sft = F.softmin(place, dim=0)
        margin_sft = F.softmin(margin, dim=0)
        bosted_sft = F.softmin(margin.float()**2, dim=0)
        bsp_prob = 1/bsp
        df_out.loc[data.index, 'place_sft'] = place_sft.tolist()
        df_out.loc[data.index, 'margin_sft'] = margin_sft.tolist()
        df_out.loc[data.index, 'bosted_sft'] = bosted_sft.tolist()
        df_out.loc[data.index, 'bsp_prob'] = bsp_prob.tolist()
        print(place_sft, margin_sft, bosted_sft, bsp_prob)
    return df_out


In [6]:
single_day.loc[single_day['place']==1, 'resultMargin']=0
test_df = add_classes(single_day)

914184479 (7, 155)
tensor([0.0116, 0.6327, 0.0043, 0.0016, 0.0856, 0.0315, 0.2328],
       dtype=torch.float64) tensor([1.1038e-03, 9.4267e-01, 4.0605e-04, 9.0601e-05, 4.9467e-03, 3.8525e-03,
        4.6933e-02], dtype=torch.float64) tensor([1.6308e-20, 9.9988e-01, 8.2250e-27, 6.9283e-38, 1.0708e-12, 7.2868e-14,
        1.2339e-04]) tensor([0.0533, 0.3558, 0.0526, 0.2083, 0.0909, 0.0932, 0.1220],
       dtype=torch.float64)
914184480 (8, 155)
tensor([4.2606e-03, 1.1582e-02, 1.5674e-03, 2.3262e-01, 6.3233e-01, 3.1482e-02,
        5.7661e-04, 8.5577e-02], dtype=torch.float64) tensor([7.7438e-05, 1.2113e-03, 3.6579e-05, 1.7978e-01, 8.0571e-01, 4.2280e-03,
        8.1619e-06, 8.9506e-03], dtype=torch.float64) tensor([6.2684e-38, 4.0508e-19, 3.3631e-44, 9.5349e-02, 9.0465e-01, 9.6881e-13,
        0.0000e+00, 1.4522e-09]) tensor([0.2474, 0.0436, 0.0344, 0.1887, 0.0615, 0.2420, 0.0111, 0.1471],
       dtype=torch.float64)
914184481 (8, 155)
tensor([6.3233e-01, 3.1482e-02, 2.3262e-01, 8.5577e-

In [7]:
test_df

,meetingDate,state,track,distance,raceId,raceTypeCode,raceNumber,boxNumber,rugNumber,runId,...,prev_start_price,stats_topaz,dogid,raceid,stats_cols,track_hash,place_sft,margin_sft,bosted_sft,bsp_prob
301162,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,1.0,1,915839614,...,38.1,"[0.5339285731315613, 1.0, 1.0, 1.0, 0.0, 0.0, ...",549391123,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.011588,0.001104,1.630813e-20,0.053331
301163,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,2.0,2,915839615,...,11.8,"[1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.1560566574335...",491585837,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.632698,0.942668,9.998766e-01,0.355826
301164,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,3.0,3,915839616,...,21.9,"[0.7160714268684387, 3.0, 6.0, 0.0462962947785...",525211021,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.004263,0.000406,8.224966e-27,0.052632
301165,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,4.0,4,915839617,...,13.8,"[0.02500000037252903, 4.0, 1.0, 0.027777777984...",638803603,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.001568,0.000091,6.928270e-38,0.208333
301166,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,6.0,6,915839619,...,12.4,"[0.13214285671710968, 6.0, 7.0, 0.333333343267...",614057168,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.085626,0.004947,1.070791e-12,0.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301250,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,3.0,3,915839704,...,11.1,"[0.1130952388048172, 3.0, 6.0, 1.0, 0.0, 0.0, ...",638803595,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.632698,0.779699,9.045495e-01,0.281690
301251,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,4.0,4,915839705,...,29.4,"[0.5773809552192688, 4.0, 1.0, 0.8863636255264...",525210989,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.232756,0.173974,9.533881e-02,0.045455
301252,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,6.0,6,915839706,...,12.4,"[0.2916666567325592, 6.0, 1.0, 0.2159090936183...",592709125,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.004263,0.000204,2.496113e-30,0.068966
301253,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,7.0,7,915839707,...,13.3,"[0.5297619104385376, 7.0, 1.0, 0.0, 0.0, 0.0, ...",556974762,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465,0.031500,0.006746,1.437683e-10,0.312500


In [51]:
single_race.loc[single_race['place']==1, 'resultMargin']=0

In [9]:
single_race.columns

Index(['meetingDate', 'state', 'track', 'distance', 'raceId', 'raceTypeCode',
       'raceNumber', 'boxNumber', 'rugNumber', 'runId',
       ...
       'prev_race_track', 'prev_race_state', 'next_race', 'prev_BSP_price',
       'prev_start_price', 'stats_topaz', 'dogid', 'raceid', 'stats_cols',
       'track_hash'],
      dtype='object', length=155)

In [10]:
single_race.BSP

301200    11.770853
301201     7.227368
301202     6.449292
301203     5.134301
301204    36.000000
301205     3.052785
301206    38.000000
301207    23.000000
Name: BSP, dtype: float64

In [53]:
place = torch.tensor(single_race['place'].values)
place_inv = 1/place
margin = torch.tensor(single_race['resultMargin'].values)
onehot = torch.tensor(single_race['win'].values)
bsp = torch.tensor(single_race['BSP'].values)

In [43]:
margin

tensor([ 1.2500,  5.0000,  0.2500,  0.2500, 11.0000,  9.7500,  9.0000,  8.5000],
       dtype=torch.float64)

In [54]:
place_sft = F.softmin(place, dim=0)
margin_sft = F.softmin(margin, dim=0)
bosted_sft = F.softmin(margin.float()**2, dim=0)
bsp_prob = 1/bsp

In [45]:
bsp_prob

tensor([0.0850, 0.1384, 0.1551, 0.1948, 0.0278, 0.3276, 0.0263, 0.0435],
       dtype=torch.float64)

In [46]:
single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']]

,boxNumber,win,resultMargin,place,resultTime,BSP
301200,1.0,0,1.25,3.0,17.8593,11.770853
301201,2.0,0,5.00,4.0,18.0798,7.227368
301202,3.0,1,0.25,1.0,17.7900,6.449292
301203,4.0,0,0.25,2.0,17.7963,5.134301
301204,5.0,0,11.00,8.0,18.4452,36.000000
301205,6.0,0,9.75,7.0,18.3633,3.052785
301206,7.0,0,9.00,6.0,18.3318,38.000000
301207,8.0,0,8.50,5.0,18.2877,23.000000


In [47]:
data

,meetingDate,state,track,distance,raceId,raceTypeCode,raceNumber,boxNumber,rugNumber,runId,...,prev_race_track,prev_race_state,next_race,prev_BSP_price,prev_start_price,stats_topaz,dogid,raceid,stats_cols,track_hash
0,2016-04-11 00:00:00+00:00,NZ,Manawatu,375,-2146657275,C1,1,1.0,1,-2146637110,...,-1,-1,109488051.0,-1.0,-1.0,"[0.2956989109516144, 1.0, 7.0, 0.6204379796981...",724410181,-2146657275,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",150
1,2016-04-11 00:00:00+00:00,NZ,Manawatu,375,-2146657275,C1,1,2.0,2,-2146637109,...,-1,-1,1656860001.0,-1.0,-1.0,"[0.2548387050628662, 2.0, 7.0, 0.0145985400304...",638670008,-2146657275,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",150
2,2016-04-11 00:00:00+00:00,NZ,Manawatu,375,-2146657275,C1,1,3.0,3,-2146637108,...,-1,-1,107841311.0,-1.0,-1.0,"[0.6043010950088501, 3.0, 1.0, 0.0583941601216...",219770075,-2146657275,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",150
3,2016-04-11 00:00:00+00:00,NZ,Manawatu,375,-2146657275,C1,1,4.0,9,-2146637102,...,-1,-1,107841312.0,-1.0,-1.0,"[1.0, 4.0, 1.0, 0.0, 0.0, 0.0, 0.1240273863077...",-738619,-2146657275,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",150
4,2016-04-11 00:00:00+00:00,NZ,Manawatu,375,-2146657275,C1,1,5.0,5,-2146637106,...,-1,-1,108121424.0,-1.0,-1.0,"[0.7462365627288818, 5.0, 1.0, 0.6277372241020...",95840103,-2146657275,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392954,2016-04-08 00:00:00+00:00,NZ,Hatrick,305,2140960086,C45,12,4.0,4,2140980795,...,Hatrick,NZ,-1,-1.0,10.8,"[0.0, 4.0, 5.0, 0.738095223903656, 0.0, 0.0, 0...",463550039,2140960086,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",784
392955,2016-04-08 00:00:00+00:00,NZ,Hatrick,305,2140960086,C45,12,5.0,5,2140980796,...,Manawatu,NZ,-1,-1.0,18.5,"[0.6374045610427856, 5.0, 5.0, 0.5476190447807...",66970113,2140960086,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",784
392956,2016-04-08 00:00:00+00:00,NZ,Hatrick,305,2140960086,C45,12,6.0,6,2140980797,...,Hatrick,NZ,-1,-1.0,6.7,"[0.5019084215164185, 6.0, 4.0, 0.0, 0.0, 0.0, ...",98640632,2140960086,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",784
392957,2016-04-08 00:00:00+00:00,NZ,Hatrick,305,2140960086,C45,12,7.0,7,2140980798,...,Hatrick,NZ,-1,-1.0,12.3,"[0.0, 7.0, 4.0, 0.3452380895614624, 0.0, 0.0, ...",463560038,2140960086,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",784


In [48]:
import plotly.graph_objects as go

# Assuming you have your data in the following format
dogs = ['Dog1', 'Dog2', 'Dog3', 'Dog4', 'Dog5', 'Dog6', 'Dog7', 'Dog8']
target_types = ['OneHot', 'place_sft', 'margin_sft', 'bsp_prob','boosted']

# Random data for demonstration
data = torch.stack([onehot,place_sft, margin_sft, bsp_prob,bosted_sft], dim=0)

# Create a 2D array of text
text = data.numpy().round(3).astype(str)

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
                   z=data,
                   x=dogs,
                   y=target_types,
                   text=text,
                   hoverinfo='text',
                   texttemplate="%{text}",
                    textfont={"size":12},
                   colorscale=[[0, 'white'], [1, 'blue']],
                   showscale=False))

# fig.update_layout(height=800, width=800, title_text="Target Types for Each Dog")

fig.show(text_auto=True,)

In [49]:
single_day

,meetingDate,state,track,distance,raceId,raceTypeCode,raceNumber,boxNumber,rugNumber,runId,...,prev_race_track,prev_race_state,next_race,prev_BSP_price,prev_start_price,stats_topaz,dogid,raceid,stats_cols,track_hash
301162,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,1.0,1,915839614,...,Addington,NZ,918532419.0,55.000000,38.1,"[0.5339285731315613, 1.0, 1.0, 1.0, 0.0, 0.0, ...",549391123,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301163,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,2.0,2,915839615,...,Addington,NZ,920247464.0,14.500000,11.8,"[1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.1560566574335...",491585837,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301164,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,3.0,3,915839616,...,Addington,NZ,914237106.0,34.995636,21.9,"[0.7160714268684387, 3.0, 6.0, 0.0462962947785...",525211021,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301165,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,4.0,4,915839617,...,Addington,NZ,914237107.0,18.000000,13.8,"[0.02500000037252903, 4.0, 1.0, 0.027777777984...",638803603,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301166,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184479,C1,1,6.0,6,915839619,...,Addington,NZ,918532429.0,17.000000,12.4,"[0.13214285671710968, 6.0, 7.0, 0.333333343267...",614057168,914184479,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301250,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,3.0,3,915839704,...,Addington,NZ,914186722.0,15.409545,11.1,"[0.1130952388048172, 3.0, 6.0, 1.0, 0.0, 0.0, ...",638803595,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301251,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,4.0,4,915839705,...,Addington,NZ,918532419.0,65.000000,29.4,"[0.5773809552192688, 4.0, 1.0, 0.8863636255264...",525210989,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301252,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,6.0,6,915839706,...,Addington,NZ,914237107.0,18.500000,12.4,"[0.2916666567325592, 6.0, 1.0, 0.2159090936183...",592709125,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465
301253,2023-07-03 00:00:00+00:00,NZ,Addington,295,914184490,C1,12,7.0,7,915839707,...,Ascot Park,NZ,914186229.0,11.500000,13.3,"[0.5297619104385376, 7.0, 1.0, 0.0, 0.0, 0.0, ...",556974762,914184490,"['dogAgeScaled', 'boxNumber', 'incomingGrade_v...",465


In [27]:
test_df.to_csv('single_day_for_viz.csv')

In [29]:
import dashtools

In [26]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go
from dash.dependencies import Input, Output

# Assuming you have your data in the following format
dogs = ['Dog1', 'Dog2', 'Dog3', 'Dog4', 'Dog5', 'Dog6', 'Dog7', 'Dog8']
target_types = ['OneHot', 'place_sft', 'margin_sft', 'bsp_prob']

# Random data for demonstration
df = test_df
race_ids = df['raceId'].unique()
data = torch.stack([onehot,place_sft, margin_sft, bsp_prob], dim=0)
result_time = torch.tensor(single_race['resultMargin'].values)

# Create a 2D array of text
text = data.numpy().astype(str)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='race-selector',
        options=[{'label': f'Race {i+1}', 'value': i} for i in race_ids],  # replace 10 with the number of races
        value=0
    ),
    dash_table.DataTable(id='table'),
    dcc.Graph(id='column-chart'),
    dcc.Graph(id='heatmap')
])

@app.callback(
    [Output('table', 'columns'), Output('table', 'data'), Output('column-chart', 'figure'), Output('heatmap', 'figure')],
    [Input('race-selector', 'value')]
)


def update_content(race_index):
    # Get data for the selected race
    single_race = df[df['raceId'] == race_index]

    # Ensure we have data for all 8 dogs
    dogs = [f'Dog{i+1}' for i in range(8)]
    
    # Prepare data for heatmap
    data_cols = ['place_sft', 'margin_sft', 'bosted_sft', 'bsp_prob']
    data = np.zeros((len(data_cols), 8))  # Initialize with zeros for 8 dogs
    
    for i, row in single_race.iterrows():
        dog_index = int(row['boxNumber']) - 1  # Assuming boxNumber starts from 1
        data[:, dog_index] = row[data_cols]

    # Prepare data for bar chart
    result_time = single_race['resultMargin'].values
    
    return (
        [{"name": i, "id": i} for i in single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].columns],
        single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].round(3).to_dict('records'),
        {
            'data': [
                go.Bar(
                    x=result_time,
                    y=dogs,
                    orientation='h',
                )
            ]
        },
        {
            'data': [
                go.Heatmap(
                    z=data,
                    x=dogs,
                    y=data_cols,
                    text=data.round(3),
                    hoverinfo='text',
                    colorscale=[[0, 'white'], [1, 'blue']],
                    showscale=False
                )
            ]
        }
    )

# Use the 'jupyterlab' mode when running in Jupyter
app.

In [39]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go
from dash.dependencies import Input, Output

# Assuming you have your data in the following format
dogs = ['Dog1', 'Dog2', 'Dog3', 'Dog4', 'Dog5', 'Dog6', 'Dog7', 'Dog8']
target_types = ['OneHot', 'place_sft', 'margin_sft', 'bsp_prob']

# Random data for demonstration
data = torch.stack([onehot,place_sft, margin_sft, bsp_prob], dim=0)
result_time = torch.tensor(single_race['resultMargin'].values)

# Create a 2D array of text
text = data.numpy().astype(str)

app = dash.Dash(__name__)

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].columns],
        data=single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].round(3).to_dict('records'),
    ),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': i, 'value': i} for i in single_race[['win','resultMargin','place','resultTime','BSP']].columns],
        value='resultMargin'
    ),
    dcc.Graph(id='column-chart'),
    dcc.Graph(
        id='heatmap',
        figure={
            'data': [
                go.Heatmap(
                    z=data,
                    x=dogs,
                    y=target_types,
                    text=text,
                    hoverinfo='text',
                    texttemplate="%{text:.3f}",
                    textfont={"size":12},
                    colorscale=[[0, 'white'], [1, 'blue']],
                    showscale=False
                )
            ]
        }
    )
])

@app.callback(
    Output('column-chart', 'figure'),
    [Input('dropdown', 'value')]
)
def update_column_chart(selected_value):
    return {
        'data': [
            go.Bar(
                x=single_race[selected_value],
                y=dogs,
                orientation='h',
            )
        ]
    }

# Use the 'jupyterlab' mode when running in Jupyter
app.run_server(mode='jupyterlab')

In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import torch

# Assuming you have your data in the following format
dogs = ['Dog1', 'Dog2', 'Dog3', 'Dog4', 'Dog5', 'Dog6', 'Dog7', 'Dog8']
target_types = ['OneHot', 'place_sft', 'margin_sft', 'bsp_prob']

# Assuming these variables are defined elsewhere in your code
# onehot, place_sft, margin_sft, bsp_prob

# Stack the data
data = torch.stack([onehot, place_sft, margin_sft, bsp_prob], dim=0)

# Assuming single_race DataFrame is already defined

# Create a 2D array of text
text = data.numpy().astype(str)

# Create subplots: 2 rows, 2 columns
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=("<b>Race Data</b>", "<b>Result Margin</b>", "<b>Target Type Heatmap</b>", ""),
                    specs=[[{"type": "table"}, {"type": "bar"}],
                           [{"type": "heatmap", "colspan": 2}, None]],
                    vertical_spacing=0.1,
                    horizontal_spacing=0.05,
                    row_heights=[0.4, 0.6])

# Add table
fig.add_trace(
    go.Table(
        header=dict(values=[f"<b>{col}</b>" for col in single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].columns],
                    fill_color='#1f77b4',
                    font=dict(color='white'),
                    align='left'),
        cells=dict(values=[single_race[col].round(3) for col in single_race[['boxNumber','win','resultMargin','place','resultTime','BSP']].columns],
                   fill_color=['#f0f0f0', 'white'],
                   align='left')
    ),
    row=1, col=1
)

# Add column chart (using 'resultMargin' as default)
fig.add_trace(
    go.Bar(
        x=single_race['resultMargin'],
        y=dogs,
        orientation='h',
        text=single_race['resultMargin'].round(3),
        textposition='outside',
        marker_color='#1f77b4',
        hoverinfo='text+y'
    ),
    row=1, col=2
)

# Add heatmap
fig.add_trace(
    go.Heatmap(
        z=data.numpy(),
        x=dogs,
        y=target_types,
        text=text,
        hoverinfo='text+x+y',
        texttemplate="%{text:.3f}",
        textfont={"size":10},
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(len=0.6, y=0.3, thickness=20)
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=1000, 
    width=1400, 
    title_text="<b>Dog Race Analysis</b>",
    font=dict(family="Arial, sans-serif"),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=24,
    showlegend=False
)

# Update axes
fig.update_xaxes(title_text="<b>Result Margin</b>", row=1, col=2, showgrid=True, gridcolor='lightgrey')
fig.update_yaxes(title_text="", row=1, col=2, showticklabels=False, showgrid=False)  # Remove y-axis
fig.update_xaxes(title_text="<b>Dogs</b>", row=2, col=1)
fig.update_yaxes(title_text="<b>Target Types</b>", row=2, col=1)

# Add hover templates
fig.update_traces(
    hovertemplate="<b>%{y}</b><br>Result Margin: %{x:.3f}<extra></extra>",
    selector=dict(type="bar")
)

# Generate HTML with some custom CSS for a modern look
html_string = f"""
<html>
    <head>
        <meta charset="utf-8" />
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f0f0f0;
            }}
            .chart-container {{
                background-color: white;
                border-radius: 8px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
                padding: 20px;
                margin-bottom: 20px;
                max-width: 1400px;
                margin-left: auto;
                margin-right: auto;
            }}
        </style>
    </head>
    <body>
        <div class="chart-container">
            {fig.to_html(full_html=False, include_plotlyjs=False)}
        </div>
    </body>
</html>
"""

# Write to file
with open('dog_race_analysis.html', 'w') as f:
    f.write(html_string)

print("HTML file 'dog_race_analysis.html' has been created.")


fig.show()

HTML file 'dog_race_analysis.html' has been created.
